# Natural Language Processing and Information Extraction
## Deep learning - practical session

__Nov 10, 2023__

__Varvara Arzt__

The material of this lecture was created by __Ádám Kovács__ and extended by Varvara Arzt


<a href="https://colab.research.google.com/github/tuw-nlp-ie/tuw-nlp-ie-2023WS/blob/main/lectures/05_Deep_learning_practical_lesson//deep_learning_practical_lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

During this lecture we are going to use a classification dataset from a shared task: [SemEval 2019 - Task 6](https://github.com/ZeyadZanaty/offenseval).

SemEval (Semantic Evaluation) is a series of international NLP research workshops whose mission is to advance the current SOTA in semantic analysis. [Here](https://semeval.github.io/SemEval2024/tasks.html) you can find a list of shared tasks for SemEval-2024.

The [SemEval 2019 - Task 6](https://github.com/ZeyadZanaty/offenseval) is about Identifying and Categorizing Offensive Language in Social Media.
__Preparation:__
- You will need the Semeval dataset (we will have code to download it)
- You will need to install pytorch:
    - pip install torch (for details on PyTorch installation depending on your OS, CUDA version etc. check the official [PyTorch documentation](https://pytorch.org/get-started/locally/))
- You will also need to have pandas, torchtext, numpy and scikit learn installed.

We are going to use an open source library for building optimized deep learning models that can be run on GPUs, the library is called [Pytorch](https://pytorch.org/docs/stable/index.html). It is one of the most widely used libraries for building neural networks/deep learning models.

In this lecture we are mostly using pure PyTorch models, but there are multiple libraries available to make it even easier to build neural networks. You are free to use them in your projects.
Just to name a few:
- TorchText: https://pytorch.org/text/stable/index.html
- AllenNLP: https://github.com/allenai/allennlp
- HuggingFace: https://huggingface.co

__NOTE: It is advised to use Google Colab for this laboratory for free access to GPUs, and also for reproducibility.__

In [ ]:
!pip install torch
!pip install pandas
!pip install scikit-learn

In [ ]:
!conda info

In [ ]:
# Import the needed libraries
import pandas as pd
import numpy as np

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

## Download the dataset and load it into a pandas DataFrame

In [ ]:
import os

if not os.path.isdir("./data"):
    os.mkdir("./data")

import urllib.request

u = urllib.request.URLopener()
u.retrieve(
    "https://raw.githubusercontent.com/ZeyadZanaty/offenseval/master/datasets/training-v1/offenseval-training-v1.tsv",
    "data/offenseval.tsv",
)

## Read in the dataset into a Pandas DataFrame
Use `pd.read_csv` with the correct parameters to read in the dataset. If done correctly, `DataFrame` should have 5 columns,
`id`, `tweet`, `subtask_a`, `subtask_b`, `subtask_c`.

In [ ]:
def read_dataset():
    train_data = pd.read_csv("./data/offenseval.tsv", sep="\t")
    return train_data

In [ ]:
train_data_unprocessed = read_dataset()
train_data_unprocessed

## Convert `subtask_a` into a binary label
The task is to classify the given tweets into two category: _offensive(OFF)_ , _not offensive (NOT)_. For machine learning algorithms you will need integer labels instead of strings. Add a new column to the dataframe called `label`, and transform the `subtask_a` column into a binary integer label.

In [ ]:
def transform(train_data):
    labels = {"NOT": 0, "OFF": 1}

    train_data["label"] = [labels[item] for item in train_data.subtask_a]
    train_data["tweet"] = train_data["tweet"].str.replace("@USER", "")

    return train_data

In [ ]:
train_data = transform(train_data_unprocessed)

In [ ]:
train_data

In [ ]:
train_data.groupby("subtask_a").size()

## Train a simple neural network on this dataset

In this notebook we are going to build different neural architectures on the task:
- A simple one layered feed forward neural network (FNN) with one-hot encoded vectors
- Adding more layers to the FNN, making it a deep neural network
- Instead of using one-hot encoded vectors we are going to add embedding vectors to the architecture, that takes the sequential nature of natural texts into account
- Then we will train LSTM networks
- At last, we will also build a Transformer architecture, that currently achieves SOTA results on a lot of tasks

First we will build one-hot-encoded vectors for each sentence, and then use a simple feed forward neural network to predict the correct labels.

In [ ]:
# First we need to import pytorch and set a fixed random seed number for reproducibility
import torch

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Split the dataset into a train and a validation dataset
Use the random seed for splitting. You should split the dataset into 70% training data and 30% validation data

In [ ]:
from sklearn.model_selection import train_test_split as split


def split_data(train_data, random_seed):
    tr_data, val_data = split(train_data, test_size=0.3, random_state=SEED)
    return tr_data, val_data

In [ ]:
tr_data, val_data = split_data(train_data, SEED)

### Use CountVectorizer to prepare the features for the sentences
_CountVectorizer_ is a great tool from _sklearn_ that helps us with basic preprocessing steps. It has lots of parameters to play with, you can check the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). It will:
- Tokenize, lowercase the text
- Filter out stopwords
- Convert the text into one-hot encoded vectors
- Select the _n_-best features

We fit CountVectorizer using _3000_ features

We will also _lemmatize_ texts using the _nltk_ package and its lemmatizer. Check the [docs](https://www.nltk.org/_modules/nltk/stem/wordnet.html) for more.

In [ ]:
!pip install nltk

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

import nltk

nltk.download("punkt")
nltk.download("wordnet")

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


def prepare_vectorizer(tr_data):
    #max_features: build a vocabulary that only considers the top max_features ordered by term frequency across the corpus.
    vectorizer = CountVectorizer(
        max_features=3000, tokenizer=LemmaTokenizer(), stop_words="english"
    )

    word_to_ix = vectorizer.fit(tr_data.tweet)

    return word_to_ix

In [ ]:
word_to_ix = prepare_vectorizer(tr_data)
# The vocab size is the length of the vocabulary, or the length of the feature vectors
VOCAB_SIZE = len(word_to_ix.vocabulary_)
assert VOCAB_SIZE == 3000

CountVectorizer can directly transform any sentence into a Bag-of-Words representation based on the corpus it is built on (Bag-of-Words representation of each document=accumulated one-hot vectors for all words in a document). It creates a document-term matrix with each row representing a document and each column addressing a token: CountVectorizer simply counts the number of times a token shows up in the document and uses this value as its weight

![onehot](https://miro.medium.com/max/886/1*_da_YknoUuryRheNS-SYWQ.png)
![bagofwords](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*3IACMnNpwVlCl8kSTJocPA.png)

In [ ]:
word_to_ix.transform(["Hello my name is varya"]).toarray()

In [ ]:
type(word_to_ix.transform(["Hello my name is varya"]))

In [ ]:
# Initialize the correct device
# It is important that every array should be on the same device or the training won't work
# A device could be either the cpu or the gpu if it is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Prepare the DataLoader for batch processing

The __prepare_dataloader(..)__ function will take the training and the validation dataset and convert them to one-hot encoded vectors with the help of the initialized CountVectorizer.

We prepare two FloatTensors and LongTensors for the converted tweets and labels of the training and the validation data.

Then zip together the vectors with the labels as a list of tuples!

In [ ]:
# Preparing the data loaders for the training and the validation sets
# PyTorch operates on it's own datatype which is very similar to numpy's arrays
# They are called Torch Tensors: https://pytorch.org/docs/stable/tensors.html
# They are optimized for training neural networks
def prepare_dataloader(tr_data, val_data, word_to_ix):
    # First we transform the tweets into one-hot encoded vectors
    # Then we create Torch Tensors from the list of the vectors
    # It is also inportant to send the Tensors to the correct device
    # All of the tensors should be on the same device when training
    tr_data_vecs = torch.FloatTensor(word_to_ix.transform(tr_data.tweet).toarray()).to(
        device
    )
    tr_labels = torch.LongTensor(tr_data.label.tolist()).to(device)

    val_data_vecs = torch.FloatTensor(
        word_to_ix.transform(val_data.tweet).toarray()
    ).to(device)
    val_labels = torch.LongTensor(val_data.label.tolist()).to(device)

    tr_data_loader = [(sample, label) for sample, label in zip(tr_data_vecs, tr_labels)]
    val_data_loader = [
        (sample, label) for sample, label in zip(val_data_vecs, val_labels)
    ]

    return tr_data_loader, val_data_loader

In [ ]:
tr_data_loader, val_data_loader = prepare_dataloader(tr_data, val_data, word_to_ix)

- __We have the correct lists now, it is time to initialize the DataLoader objects!__
- __Create two DataLoader objects with the lists we have created__
- __Shuffle the training data but not the validation data!__

In [ ]:
# We then define a BATCH_SIZE for our model
# Usually we don't feed the whole dataset into our model at once
# For this we have the BATCH_SIZE parameter
# Try to experiment with different sized batches and see if changing this will improve the performance or not!
BATCH_SIZE = 64

In [ ]:
from torch.utils.data import DataLoader

# The DataLoader(https://pytorch.org/docs/stable/data.html) class helps us to prepare the training batches
# It has a lot of useful parameters, one of it is _shuffle_ which will randomize the training dataset in each epoch
# This can also improve the performance of our model
def create_dataloader_iterators(tr_data_loader, val_data_loader, BATCH_SIZE):
    train_iterator = DataLoader(
        tr_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    valid_iterator = DataLoader(
        val_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    return train_iterator, valid_iterator

In [ ]:
train_iterator, valid_iterator = create_dataloader_iterators(
    tr_data_loader, val_data_loader, BATCH_SIZE
)
assert type(train_iterator) == torch.utils.data.dataloader.DataLoader

In [ ]:
valid_iterator

### Building the first PyTorch model
At first, the model will contain a single Linear layer that takes bag-of-words vectors and transforms it into the dimension of the __NUM_LABELS__(how many classes we are trying to predict). Then, run through the output on a softmax activation to produce probabilites of the classes!

In [ ]:
from torch import nn
import torch.nn.functional as F


class BoWClassifier(nn.Module):  # inheriting from nn.Module!
    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.
        # Torch defines nn.Linear(), which provides the affine map.
        # Note that we could add more Linear Layers here connected to each other
        # Then we would also need to have a HIDDEN_SIZE hyperparameter as an input to our model
        # Then, with activation functions between them (e.g. RELU) we could have a "Deep" model
        # This is just an example for a shallow network
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec, sequence_lens):
        # Ignore sequence_lens for now!
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        # Softmax will provide a probability distribution among the classes
        # We can then use this for our loss function
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [ ]:
# The INPUT_DIM is the size of our input vectors
INPUT_DIM = VOCAB_SIZE
# We have only 2 classes
OUTPUT_DIM = 2

In [ ]:
# Init the model
# At first it is untrained, the weights are assigned random
model = BoWClassifier(OUTPUT_DIM, INPUT_DIM)

In [ ]:
# Set the optimizer and the loss function!
# https://pytorch.org/docs/stable/optim.html
import torch.optim as optim

# The optimizer will update the weights of our model based on the loss function
# This is essential for correct training
# The _lr_ parameter is the learning rate
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()

In [ ]:
# Copy the model and the loss function to the correct device
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
assert model.linear.out_features == 2

### Training and evaluating PyTorch models
- __calculate_performance__: This should calculate the batch-wise precision, recall, and fscore of your model!
- __train__ - Train your model on the training data! This function should set the model to training mode, then use the given iterator to iterate through the training samples and make predictions using the provided model. You should then propagate back the error with the loss function and the optimizer. Finally return the average epoch loss and performance!
- __evaluate__ - Evaluate your model on the validation dataset. This function is essentially the same as the trainnig function, but you should set your model to eval mode and don't propagate back the errors to your weights!

In [ ]:
from sklearn.metrics import precision_recall_fscore_support


def calculate_performance(preds, y):
    """
    Returns precision, recall, fscore per batch
    """
    # Get the predicted label from the probabilities
    rounded_preds = preds.argmax(1)

    # Calculate the correct predictions batch-wise and calculate precision, recall, and fscore
    # WARNING: Tensors here could be on the GPU, so make sure to copy everything to CPU
    precision, recall, fscore, support = precision_recall_fscore_support(
        rounded_preds.cpu(), y.cpu()
    )

    return precision[1], recall[1], fscore[1]

In [ ]:
import torch.nn.functional as F


def train(model, iterator, optimizer, criterion):
    # We will calculate loss and accuracy epoch-wise based on average batch accuracy
    epoch_loss = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_fscore = 0

    # You always need to set your model to training mode
    # If you don't set your model to training mode the error won't propagate back to the weights
    model.train()

    # We calculate the error on batches so the iterator will return matrices with shape [BATCH_SIZE, VOCAB_SIZE]
    for batch in iterator:
        text_vecs = batch[0]
        labels = batch[1]
        sen_lens = []
        texts = []

        # This is for later!
        if len(batch) > 2:
            sen_lens = batch[2]
            texts = batch[3]

        # We reset the gradients from the last step, so the loss will be calculated correctly (and not added together)
        optimizer.zero_grad()

        # This runs the forward function on your model (you don't need to call it directly)
        predictions = model(text_vecs, sen_lens)

        # Calculate the loss and the accuracy on the predictions (the predictions are log probabilities, remember!)
        loss = criterion(predictions, labels)

        prec, recall, fscore = calculate_performance(predictions, labels)

        # Propagate the error back on the model (this means changing the initial weights in your model)
        # Calculate gradients on parameters that requries grad
        loss.backward()
        # Update the parameters
        optimizer.step()

        # We add batch-wise loss to the epoch-wise loss
        epoch_loss += loss.item()
        # We also do the same with the scores
        epoch_prec += prec.item()
        epoch_recall += recall.item()
        epoch_fscore += fscore.item()
    return (
        epoch_loss / len(iterator),
        epoch_prec / len(iterator),
        epoch_recall / len(iterator),
        epoch_fscore / len(iterator),
    )

In [ ]:
# The evaluation is done on the validation dataset
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_fscore = 0
    # On the validation dataset we don't want training so we need to set the model on evaluation mode
    model.eval()

    # Also tell Pytorch to not propagate any error backwards in the model or calculate gradients
    # This is needed when you only want to make predictions and use your model in inference mode!
    with torch.no_grad():

        # The remaining part is the same with the difference of not using the optimizer to backpropagation
        for batch in iterator:
            text_vecs = batch[0]
            labels = batch[1]
            sen_lens = []
            texts = []

            if len(batch) > 2:
                sen_lens = batch[2]
                texts = batch[3]

            predictions = model(text_vecs, sen_lens)
            loss = criterion(predictions, labels)

            prec, recall, fscore = calculate_performance(predictions, labels)

            epoch_loss += loss.item()
            epoch_prec += prec.item()
            epoch_recall += recall.item()
            epoch_fscore += fscore.item()

    # Return averaged loss on the whole epoch!
    return (
        epoch_loss / len(iterator),
        epoch_prec / len(iterator),
        epoch_recall / len(iterator),
        epoch_fscore / len(iterator),
    )

In [ ]:
import time

# This is just for measuring training time!
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Training loop!
Below is the training loop of our model! Try to set an EPOCH number that will correctly train your model :) (it is not underfitted but neither overfitted!

In [ ]:
def training_loop(epoch_number=15):
    # Set an EPOCH number!
    N_EPOCHS = epoch_number

    best_valid_loss = float("inf")

    # We loop forward on the epoch number
    for epoch in range(N_EPOCHS):

        start_time = time.time()

        # Train the model on the training set using the dataloader
        train_loss, train_prec, train_rec, train_fscore = train(
            model, train_iterator, optimizer, criterion
        )
        # And validate your model on the validation set
        valid_loss, valid_prec, valid_rec, valid_fscore = evaluate(
            model, valid_iterator, criterion
        )

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # If we find a better model, we save the weights so later we may want to reload it
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), "tut1-model.pt")

        print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(
            f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
        )
        print(
            f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
        )

In [ ]:
training_loop()


__NOTE: DON'T FORGET TO RERUN THE MODEL INITIALIZATION WHEN YOU ARE TRYING TO RUN THE MODEL MULTIPLE TIMES. IF YOU DON'T REINITIALIZE THE MODEL IT WILL CONTINUE THE TRAINING WHERE IT HAS STOPPED LAST TIME AND DOESN'T RUN FROM SRATCH!__

These lines:

```python
model = BoWClassifier(OUTPUT_DIM, INPUT_DIM)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()
model = model.to(device)
criterion = criterion.to(device)
```

This will reinitialize the model!

In [ ]:
def reinitialize(model):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.NLLLoss()
    model = model.to(device)
    criterion = criterion.to(device)

In [ ]:
reinitialize(BoWClassifier(OUTPUT_DIM, INPUT_DIM))

## Add more linear layers to the model and experiment with other hyper-parameters

### More layers

Currently we only have a single linear layers in our model. We are now adding more linear layers to the model.
We also introduce a HIDDEN_SIZE parameter that will be the size of the intermediate representation between the linear layers. Also adding a RELU activation function between the linear layers.

See more:
- https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
- https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_nn.html

In [ ]:
from torch import nn


class BoWDeepClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size, hidden_size):
        super(BoWDeepClassifier, self).__init__()
        # First linear layer
        self.linear1 = nn.Linear(vocab_size, hidden_size)
        # Non-linear activation function between them
        self.relu = torch.nn.ReLU()
        # Second layer
        self.linear2 = nn.Linear(hidden_size, num_labels)

    def forward(self, bow_vec, sequence_lens):
        # Run the input vector through every layer
        output = self.linear1(bow_vec)
        output = self.relu(output)
        output = self.linear2(output)

        # Get the probabilities
        return F.log_softmax(output, dim=1)

In [ ]:
HIDDEN_SIZE = 200
learning_rate = 0.001
BATCH_SIZE = 64
N_EPOCHS = 15

In [ ]:
model = BoWDeepClassifier(OUTPUT_DIM, INPUT_DIM, HIDDEN_SIZE)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
training_loop()

## Implement automatic early-stopping in the training loop
Early stopping is a very easy method to avoid the overfitting of your model.

We could:
- Save the training and the validation loss of the last two epochs (if you are at least in the third epoch)
- If the loss increased in the last two epoch on the training data but descreased or stagnated in the validation data, you should stop the training automatically!

In the training loop below we do a very basic early stopping after there is no improvement in validation loss on a predefined number of consecutive epochs (=patience).

In [ ]:
reinitialize(BoWClassifier(OUTPUT_DIM, INPUT_DIM))

In [ ]:
model = BoWDeepClassifier(OUTPUT_DIM, INPUT_DIM, HIDDEN_SIZE)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def training_loop(epoch_number=15, patience=3):
    # Set an EPOCH number!
    N_EPOCHS = epoch_number
    consecutive_no_improvement = 0

    best_valid_loss = float("inf")

    # We loop forward on the epoch number
    for epoch in range(N_EPOCHS):

        start_time = time.time()

        # Train the model on the training set using the dataloader
        train_loss, train_prec, train_rec, train_fscore = train(
            model, train_iterator, optimizer, criterion
        )
        # And validate your model on the validation set
        valid_loss, valid_prec, valid_rec, valid_fscore = evaluate(
            model, valid_iterator, criterion
        )

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # If we find a better model, we save the weights so later we may want to reload it
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            consecutive_no_improvement = 0
            torch.save(model.state_dict(), "tut1-model.pt")

        else:
            consecutive_no_improvement += 1

        print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(
            f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
        )
        print(
            f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
        )

        # Check for early stopping
        if consecutive_no_improvement >= patience:
            print(f"No improvement in validation loss for {patience} consecutive epochs. Early stopping after epoch {epoch+1}.")
            break  # Terminate training loop

    return best_valid_loss

In [ ]:
training_loop()

## Handling class imbalance
Our data is imbalanced, the first class has twice the population of the second class.

One way of handling imbalanced data is to weight the loss function, so it penalizes errors on the smaller class.

Look at the documentation of the loss function: https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html

Set the weights based on the inverse population of the classes (so the less sample a class has, more the errors will be penalized!)

In [ ]:
tr_data.groupby("label").size()

In [ ]:
weights = torch.Tensor([1, 2])
criterion = nn.NLLLoss(weight=weights)

## Adding an Embedding Layer to the network

- We only used Bag-of-Words representations as our features until now
- Now we will introduce an [embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) layer into our network.
- We will feed the words into our network one-by-one, and the layer will learn a dense vector representation for each word

![embeddingbag](https://pytorch.org/tutorials/_images/text_sentiment_ngrams_model.png)

_from pytorch.org_

In [ ]:
# Get the analyzer to get the word-id mapping from CountVectorizer
an = word_to_ix.build_analyzer() #handles preprocessing, tokenisation and removal of stop words

In [ ]:
an("hello my name is varya")

Get the key with maximum values in a dictionary (keys are tokens; values are number of occurrences):

`word_to_ix.vocabulary_` returns mapping of terms (tokens) to feature indices (number of occurrences)

In [ ]:
max(word_to_ix.vocabulary_, key=word_to_ix.vocabulary_.get)

As we remember, we passed a parameter `max_features=3000` to a CountVectorizer, so our vocabulary size is 3000.

In [ ]:
len(word_to_ix.vocabulary_)

In [ ]:
def create_input(dataset, analyzer, vocabulary):
    dataset_as_indices = []

    # We go through each tweet in the dataset
    # We need to add two additional symbols to the vocabulary
    # We have 3000 features, ranged 0-2999
    # We add 3000 as an id for the "unknown" words not among the features
    # 3001 will be the symbol for padding, but about this later!
    for tweet in dataset:
        tokens = analyzer(tweet)
        token_ids = []

        for token in tokens:
            # if the token is in the vocab, we add the id
            if token in vocabulary:
                token_ids.append(vocabulary[token])
            # else we add the id of the unknown token
            else:
                token_ids.append(3000)

        # if we removed every token during preprocessing (stopword removal, lemmatization), we add the unknown token to the list so it won't be empty
        if not token_ids:
            token_ids.append(3000)
        dataset_as_indices.append(torch.LongTensor(token_ids).to(device))

    return dataset_as_indices

In [ ]:
tr_data.tweet.str.len()

In [ ]:
len(tr_data)

In [ ]:
len(val_data)

In [ ]:
# We add the length of the tweets so sentences with similar lengths will be next to each other
# This can be important because of padding
tr_data["length"] = tr_data.tweet.str.len()
val_data["length"] = val_data.tweet.str.len()

In [ ]:
tr_data.tweet.str.len()

In [ ]:
tr_data = tr_data.sort_values(by="length")
val_data = val_data.sort_values(by="length")

In [ ]:
# We create the dataset as ids of tokens
dataset_as_ids = create_input(tr_data.tweet, an, word_to_ix.vocabulary_)

Actually, when experimenting with different indices, you will see that many tensors that give a numeric representation of each tweet contain only one element. This is because of removal of stop words:

In [ ]:
dataset_as_ids[145]

Many of them contain only unknown token with an index `3000` (so, it was really important to add it, otherwise our tensor would be empty):

In [ ]:
dataset_as_ids[17]

### Padding

- We didn't need to take care of input padding when using Bag-of-Words vectors
- Padding handles different sized inputs
- We can pad sequences from the left, or from the right

![padding](https://miro.medium.com/max/1218/1*zsIXWoN0_CE9PXzmY3tIjQ.png)

_image from https://towardsdatascience.com/nlp-preparing-text-for-deep-learning-model-using-tensorflow2-461428138657_

`torch.nn.utils.rnn.pad_sequence` takes a list of tensors as input (sequences), calculates max length, and then pads all the sequences with `padding_value` to make them all have the same length

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# pad_sequence will take care of the padding
# we will need to provide a padding_value to it
padded = pad_sequence(dataset_as_ids, batch_first=True, padding_value=3001)
padded

Here you can see the dimension of `padded`, so max length among the sequences is 62.

In [ ]:
padded.shape

In [ ]:
def prepare_dataloader_with_padding(tr_data, val_data, word_to_ix):
    # First create the id representations of the input vectors
    # Then pad the sequences so all of the input is the same size
    # We padded texts for the whole dataset, this could have been done batch-wise also!
    tr_data_vecs = pad_sequence(
        create_input(tr_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True,
        padding_value=3001,
    )
    tr_labels = torch.LongTensor(tr_data.label.tolist()).to(device)
    tr_lens = torch.LongTensor(
        [len(i) for i in create_input(tr_data.tweet, an, word_to_ix.vocabulary_)] #len of tweets
    )
    print(tr_lens)
    print(tr_lens.shape)

    # We also add the texts to the batches
    # This is for the Transformer models, you wont need this in the next experiments
    tr_sents = tr_data.tweet.tolist()

    val_data_vecs = pad_sequence(
        create_input(val_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True,
        padding_value=3001,
    )
    val_labels = torch.LongTensor(val_data.label.tolist()).to(device)
    val_lens = torch.LongTensor(
        [len(i) for i in create_input(val_data.tweet, an, word_to_ix.vocabulary_)]
    )

    val_sents = val_data.tweet.tolist()

    tr_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            tr_data_vecs, tr_labels, tr_lens, tr_sents
        )
    ]
    val_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            val_data_vecs, val_labels, val_lens, val_sents
        )
    ]

    return tr_data_loader, val_data_loader

In [ ]:
tr_data_loader, val_data_loader = prepare_dataloader_with_padding(
    tr_data, val_data, word_to_ix
)

In [ ]:
def create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
):
    train_iterator = DataLoader(
        tr_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=True,
    )

    valid_iterator = DataLoader(
        val_data_loader,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    return train_iterator, valid_iterator

In [ ]:
train_iterator, valid_iterator = create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
)

`next(iter(train_iterator))` returns a single batch of data (=batch size)

In [ ]:
next(iter(train_iterator))

In [ ]:
samples, labels, lengths, sents = next(iter(train_iterator))
print(f'Dimension of a tensor with data samples on each iteration: {samples.shape}')
print(f'Dimension of sents (tuple) on each iteration: {len(sents)}')
print(samples[1]) #1st sample with 7 unique ids that are then padded to tensor len=62
print(lengths[1])
print(sents[1])

![embedding](https://github.com/bentrevett/pytorch-sentiment-analysis/raw/b4efbefa47672174394a8b6a27d4e7bc193bc224/assets/sentiment8.png)

_image from bentrevett_

In [ ]:
from torch import nn
import numpy as np


class BoWClassifierWithEmbedding(nn.Module):
    def __init__(self, num_labels, vocab_size, embedding_dim):
        super(BoWClassifierWithEmbedding, self).__init__()

        # We define the embedding layer here
        # It will convert a list of ids: [1, 50, 64, 2006]
        # Into a list of vectors, one for each word
        # The embedding layer will learn the vectors from the contexts
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=3001)
        # We could also load precomputed embeddings, e.g. GloVe, in some cases we don't want to train the embedding layer
        # In this case we enable the training
        self.embedding.weight.requires_grad = True

        self.linear = nn.Linear(embedding_dim, num_labels)

    def forward(self, text, sequence_lens):
        # First we create the embedded vectors
        embedded = self.embedding(text) #dim=64x62x100 (batch_size x seq_len x embedding_dim)
        #print(embedded.shape)
        #print(embedded.shape[1])
        # We need a pooling to convert a list of embedded words to a sentence vector
        # We could have chosen different pooling, e.g. min, max, average..
        # With LSTM we also do a pooling, just smarter
        pooled = F.max_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) #dim=64x100 (batch_size x embedding_dim)
        #print(pooled.shape)
        return F.log_softmax(self.linear(pooled), dim=1)

In [ ]:
INPUT_DIM = VOCAB_SIZE + 2
OUTPUT_DIM = 2
EMBEDDING_DIM = 100
HIDDEN_DIM = 20
criterion = nn.NLLLoss()

model = BoWClassifierWithEmbedding(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM)
#model = LSTMClassifier(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
training_loop(epoch_number=15)

## Standard RNNs & LSTMs

Unrolled RNN (image source: [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)):

![unrolled_rnn1](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Repeating module in the standard RNN (image source: [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)):

![rnn_module](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

Repeating module in the LSTM (image source: [colah's blog](https://colah.github.io/posts/2015-08-Understanding-LSTMs/))

![repeating_module_lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

Output of the LSTM layer..

![lstm](https://i.stack.imgur.com/SjnTl.png)

_image from stackoverflow_

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size, embedding_dim, hidden_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=3001)
        self.embedding.weight.requires_grad = True

        # Define the LSTM layer
        # Documentation: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            batch_first=True,
            num_layers=1,
            bidirectional=False,
        )
        self.linear = nn.Linear(hidden_dim, num_labels)
        # Dropout to overcome overfitting
        self.dropout = nn.Dropout(0.25)

    def forward(self, text, sequence_lens):
        embedded = self.embedding(text)

        # To ensure LSTM doesn't learn gradients for the id of the padding symbol
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, sequence_lens, enforce_sorted=False, batch_first=True
        )
        packed_outputs, (h, c) = self.lstm(packed)
        # extract LSTM outputs (not used here)
        lstm_outputs, lens = nn.utils.rnn.pad_packed_sequence(
            packed_outputs, batch_first=True
        )

        # We use the last hidden vector from LSTM
        y = self.linear(h[-1])
        log_probs = F.log_softmax(y, dim=1)
        return log_probs

In [ ]:
INPUT_DIM = VOCAB_SIZE + 2
OUTPUT_DIM = 2
EMBEDDING_DIM = 100
HIDDEN_DIM = 20
criterion = nn.NLLLoss()

model = BoWClassifierWithEmbedding(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM)
#model = LSTMClassifier(OUTPUT_DIM, INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
training_loop(epoch_number=15)

## CNN for NLP

- works with input of fixed length (to handle it we need padding: fill sequences of different length with tokens that do not contain any meaning)
- kernels (filters) in the visualisation below correspond to kernels for bigrams, trigrams and and tetragrams (2 for each type of kernels)
- In Computer Vision, our filters slide over local patches of an image, but in NLP we typically use filters that slide over full rows of the matrix (words). Thus, the second dimension of our filters is usually the same as the number of columns of the input matrix

CNN architecture for NLP introduced in [Zhang et al. A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification. 2015](https://arxiv.org/pdf/1510.03820.pdf)

![cnnfornlp](https://media.arxiv-vanity.com/render-output/6570546/x1.png)

## Transformers

## Attention mechanism in a nutshell

To recap: seq2seq: encoder-decoder architecture with 2 RNNs without an attention block (image source: [medium article](https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346))

![encoder_decoder_rnn](https://miro.medium.com/v2/resize:fit:720/format:webp/1*1JcHGUU7rFgtXC_mydUA_Q.jpeg)

##### Attention mechanism and neural machine translation

Machine translation

fascinating history that started more than 100 years ago (translating machine of Troyanskii, Georgetown–IBM experiment, ...): some of these experiments were driven by the Cold War

- rule-based systems
- statistical machine translation (phrase-based statistical machine translation is SOTA until 2016)
- NN based approaches without attention mechanism
- RNN based Encoder-Decoder architecture with attention mechanism (introduced in 2014 in the paper [Bengio et al. Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)): read [this Google Blog](https://blog.research.google/2016/09/a-neural-network-for-machine.html) from 2016 on machine translation system they introduced back then, namely Google Neural Machine Translation (GNMT) that contained 8 RNN layers in the encoder and decoder + attention mechanism
- Attention is all you need :D


![attention_encoder_decoder](https://miro.medium.com/v2/resize:fit:640/format:webp/1*KrCUK30Y1R7TV7xE-ltcpg.png)

Image source: [medium article](https://medium.com/data-science-community-srm/understanding-encoders-decoders-with-attention-based-mechanism-c1eb7164c581)

To completely understand the transformers architecture look at this lecture held by Judit Acs (on the course of Introduction to Python and Natural Language Technologies in BME):
- https://github.com/bmeaut/python_nlp_2021_spring/blob/main/lectures/09_Transformers_BERT.ipynb

Here I will only include and present the necessary details _from the lecture_ about transformers and BERT.

### Problems with recurrent neural networks:

Recall that we used recurrent neural cells, specifically LSTMs to encode a list of vectors into a sentence vector.

- Problem 1. No parallelism

        - LSTMs are recurrent, they rely on their left and right history, so the symbols need to be processed in order -> no parallelism.

- Problem 2. Long-range dependencies

        - Long-range dependencies are not infrequent in NLP.

        - "The people/person who called and wanted to rent your house when you go away next year are/is from California" -- Miller & Chomsky 1963

        - LSTMs have a problem capturing these because there are too many backpropagation steps between the symbols.

## Transformers

Good explanation of transformer architecture: https://jalammar.github.io/illustrated-transformer/

![transformer_architecture](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png)

Image source: https://machinelearningmastery.com

- Attention is all you need: no convolutions or recurrences are used. Introduction of multi-headed self-attention. 

"The animal didn't cross the street because it was too tired" -- We want self-attention to associate *it* with *animal*

![self_attention](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*maneqIvralwRZWf0g3hS5w.png)

Image source: [towardsdatascience](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021#4c16)

![self_attention_detailed](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*Rv_pntt-N2WL7LMbIptHxQ.png)

Image source: [towardsdatascience](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021#4c16)

Introduced in [Attention Is All You Need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) by Vaswani et al., 2017

Transformers solve Problem 1 by relying purely on attention instead of recurrence.

Not having recurrent connections means that sequence position no longer matters.

Recurrence is replaced by self-attention.

- Transformers are available in the __transformers__ Python package: https://github.com/huggingface/transformers.
- There are thousands of pretrained transformers models in different languages and with different architectures.
- With the huggingface package there is a unified interface to download and use all the models. Browse https://huggingface.co/models for more!
- There is also a great blog post to understand the architecture of transformers: https://jalammar.github.io/illustrated-transformer/

### BERT

[BERT](https://www.aclweb.org/anthology/N19-1423/): Pre-training of Deep Bidirectional Transformers for Language Understanding by Devlin et al. 2018, 17500 citations

[BERTology](https://huggingface.co/transformers/bertology.html) is the nickname for the growing amount of BERT-related research.

BERT trains a transformer model on two tasks:

- Masked language model:

    - 15% of the tokenswordpieces are selected at the beginning.
    - 80% of those are replaced with [MASK],
    - 10% are replaced with a random token,
    - 10% are kept intact.

- Next sentence prediction:
    - Are sentences A and B consecutive sentences?
    - Generate 50-50%.
    - Binary classification task.
    

### Training, Finetuning BERT

- BERT models are (masked-)language models that were usually trained on large corpora.

- e.g. BERT base model was trained on BookCorpus, a dataset consisting of 11,038 unpublished books and English Wikipedia.

#### Finetuning

- Get a trained BERT model.
- Add a small classification layer on top (typically 2-layer fully-connected layers).
- Train BERT along with the classification layer on an annotated dataset.
- Much smaller than the data BERT was trained on
- Another option: freeze BERT and train the classification layer only.
    - Easier training regime.
    - Smaller memory footprint.
    - Worse performance.

<img src="https://production-media.paperswithcode.com/methods/new_BERT_Overall.jpg" alt="finetune" width="800px"/>

In [ ]:
!pip install transformers

### WordPiece tokenizer
- BERT has its own tokenizer
- All inputs must be tokenized with BERT
- You don't need to remove stopwords, lemmatize, preprocess the input for BERT

- It is a middle ground between word and character tokenization.

- Static vocabulary:
    - Special tokens: [CLS], [SEP], [MASK], [UNK]
    - It tokenizes everything, falling back to characters and [UNK] if necessary

In [ ]:
!pip3 install emoji==0.6.0

In [ ]:
from transformers import BertTokenizer, AutoTokenizer

In [ ]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

In [ ]:
print(type(tokenizer))
print(len(tokenizer.get_vocab()))

tokenizer.tokenize("My full name is Varvara Andreevna.")

In [ ]:
tokenizer("There are black cats and black dogs.", "Another sentence.")

### Train a BertForSequenceClassification model on the dataset

In [ ]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, TrainingArguments, RobertaForSequenceClassification, Trainer

__BertForSequenceClassification__ is a helper class to train transformer-based BERT models. It puts a classification layer on top of a pretrained model.

Read more in the documentation: https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification

In [ ]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#_ = model.to(device)

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)
_ = model.to(device)

#model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)
#_ = model.to(device)

In [ ]:
# We only want to finetune the classification layer on top of BERT
for p in model.base_model.parameters():
    p.requires_grad = False

In [ ]:
params = list(model.named_parameters())

print(f"The BERT model has {len(params)} different named parameters.")

print("==== Embedding Layer ====\n")

for p in params[0:5]:
    print(f"{p[0]} {str(tuple(p[1].size()))}")

print("\n==== First Transformer ====\n")

for p in params[5:21]:
    print(f"{p[0]} {str(tuple(p[1].size()))}")

print("\n==== Output Layer ====\n")

for p in params[-4:]:
    print(f"{p[0]} {str(tuple(p[1].size()))}")

In [ ]:
N_EPOCHS = 5
optimizer = optim.Adam(model.parameters())

#weights = torch.Tensor([1, 2])
#criterion = nn.CrossEntropyLoss(weight=weights)

In [ ]:
tr_data_loader, val_data_loader = prepare_dataloader_with_padding(
    tr_data, val_data, word_to_ix
)

train_iterator, valid_iterator = create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
)

In [ ]:
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_epoch_loss = 0
    train_epoch_prec = 0
    train_epoch_recall = 0
    train_epoch_fscore = 0
    model.train()

    # We use our own iterator but now use the raw texts instead of the ID tokens
    for train_batch in train_iterator:
        labels = train_batch[1]
        texts = train_batch[3]

        optimizer.zero_grad()

        # We use BERT's own tokenizer on raw texts
        # Check the documentation: https://huggingface.co/transformers/main_classes/tokenizer.html
        encoded = tokenizer(
            texts,
            truncation=True,
            max_length=128,
            padding=True,
            return_tensors="pt",
        )

        # BERT converts texts into IDs of its own vocabulary
        input_ids = encoded["input_ids"].to(device)
        # Mask to avoid performing attention on padding token indices.
        attention_mask = encoded["attention_mask"].to(device)

        # Run the model
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0] #negative log likelihood loss


        predictions = outputs[1]
        #print(predictions)
        prec, recall, fscore = calculate_performance(predictions, labels)

        loss.backward()
        optimizer.step()

        train_epoch_loss += loss.item()
        train_epoch_prec += prec.item()
        train_epoch_recall += recall.item()
        train_epoch_fscore += fscore.item()

    train_loss = train_epoch_loss / len(train_iterator)
    train_prec = train_epoch_prec / len(train_iterator)
    train_rec = train_epoch_recall / len(train_iterator)
    train_fscore = train_epoch_fscore / len(train_iterator)

    # And validate your model on the validation set
    valid_epoch_loss = 0
    valid_epoch_prec = 0
    valid_epoch_recall = 0
    valid_epoch_fscore = 0
    model.eval()

    with torch.no_grad():
        for valid_batch in valid_iterator:
            labels = valid_batch[1]
            texts = valid_batch[3]

            encoded = tokenizer(
                texts,
                truncation=True,
                max_length=128,
                padding=True,
                return_tensors="pt",
            )
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            predictions = outputs[1]
            prec, recall, fscore = calculate_performance(predictions, labels)

            # We add batch-wise loss to the epoch-wise loss
            valid_epoch_loss += loss.item()
            valid_epoch_prec += prec.item()
            valid_epoch_recall += recall.item()
            valid_epoch_fscore += fscore.item()

    valid_loss = valid_epoch_loss / len(valid_iterator)
    valid_prec = valid_epoch_prec / len(valid_iterator)
    valid_rec = valid_epoch_recall / len(valid_iterator)
    valid_fscore = valid_epoch_fscore / len(valid_iterator)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(
        f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
    )
    print(
        f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
    )

## Same experiment with HuggingFace

In this experiment we will also look at how you can track your experiments with [mlflow](https://mlflow.org)

In [ ]:
!pip install -U accelerate
!pip install -U transformers

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

In [ ]:
pip install safetensors

In [ ]:
!pip install mlflow

In [ ]:
import mlflow

Check the train and val data again

In [ ]:
tr_data

In [ ]:
val_data

In [ ]:
#convert a df column into a list of strings
train_texts = tr_data.tweet.tolist()

#convert a df column into a list of strings
train_labels = tr_data.label.tolist()

#convert a df column into a list of strings
val_texts = val_data.tweet.tolist()

#convert a df column into a list of strings
val_labels = val_data.label.tolist()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Now, we finetune our BERT model with different hyperparameters. Before we had `batch_size=64`, default batch size in the HugggingFace class `transformers.TrainingArguments`is 8. We also change the learnung rate.

##### Change of hyperparameters
- `batch_size=8`
- `learning_rate=5e-5`
##### Keep track of experiments with MLflow: see [huggingface documentation](https://huggingface.co/docs/transformers/v4.20.1/en/main_classes/callback#transformers.integrations.MLflowCallback)



Note: if you get an error similar to `ImportError: Using the Trainer with PyTorch requires accelerate>=0.20.1: Please run pip install transformers[torch] or pip install accelerate -U` when running code on the colab, please make sure you run the command `!pip install accelerate -U`. After that do the following:
1. In the top menu click **Runtime** → **Restart Runtime**
2. Do not rerun any cells with !pip install in them
3. Rerun all the other code cells that refer to the experiment using Huggingface and you should be good to go!

In [ ]:
#with mlflow.start_run():
training_args = TrainingArguments(
    output_dir='./results_bertbaseuncased_semeval_20231109',          # output directory
    num_train_epochs=5,              # total number of training epochs
    #per_device_train_batch_size=16,  # batch size per device during training
    #per_device_eval_batch_size=16,   # batch size for evaluation
    #warmup_steps=10,                # number of warmup steps for learning rate scheduler
    #weight_decay=0.1,               # strength of weight decay
    learning_rate=5e-5,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    #logging_dir='./logs_bertbaseuncased_20231109',            # directory for storing logs
    load_best_model_at_end=True,
    #logging_steps=10,
    )

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
#model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
#model.resize_token_embeddings(len(tokenizer))


# https://huggingface.co/docs/transformers/v4.20.1/en/main_classes/callback#transformers.integrations.MLflowCallback

os.environ["MLFLOW_EXPERIMENT_NAME"] = "tweet-mlflow"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
#os.environ["MLFLOW_TRACKING_URI"]=""
os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

#https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/audio_classification.ipynb#scrollTo=xc_MTm0Ks3DF
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()
#trainer.evaluate()

mlflow.end_run()

#https://discuss.huggingface.co/t/how-to-save-my-model-to-use-it-later/20568/3
trainer.save_model("/Users/varya/tu_wien_teaching_private/nlp_and_ie/tuw-nlp-ie-2023WS_my_lecture/mlflow_tracking/model_bertbaseuncased_semeval_20231109/")